# Day 3: Disney Parks Guest Spending Behavior

You are a data analyst working with the Disney Parks revenue team to understand nuanced guest spending patterns across different park experiences. The team wants to develop a comprehensive view of visitor purchasing behaviors. Your goal is to uncover meaningful insights that can drive personalized marketing strategies.

In [ ]:
import pandas as pd
import numpy as np

fct_guest_spending_data = [
  {
    "guest_id": 1,
    "visit_date": "2024-07-05",
    "amount_spent": 50,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-07-06",
    "amount_spent": 30,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-07-10",
    "amount_spent": 20.5,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 4,
    "visit_date": "2024-07-12",
    "amount_spent": 40,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-07-15",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 5,
    "visit_date": "2024-07-20",
    "amount_spent": 60,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 6,
    "visit_date": "2024-07-25",
    "amount_spent": 25,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-03",
    "amount_spent": 55,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-15",
    "amount_spent": 45,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-05",
    "amount_spent": 22,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-20",
    "amount_spent": 38,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 7,
    "visit_date": "2024-08-10",
    "amount_spent": 15,
    "park_experience_type": "Character Meet"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-25",
    "amount_spent": 28,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-27",
    "amount_spent": 32,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-02",
    "amount_spent": 65,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-05",
    "amount_spent": 50,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 9,
    "visit_date": "2024-09-15",
    "amount_spent": 40,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 10,
    "visit_date": "2024-09-20",
    "amount_spent": 70,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-25",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-28",
    "amount_spent": 10,
    "park_experience_type": "Character Meet"
  }
]
fct_guest_spending = pd.DataFrame(fct_guest_spending_data)


## Question 1

What is the average spending per guest per visit for each park experience type during July 2024? Ensure that park experience types with no recorded transactions are shown with an average spending of 0.0. This analysis helps establish baseline spending differences essential for later segmentation.

In [ ]:
# Note: pandas and numpy are already imported as pd and np
# The following tables are loaded as pandas DataFrames with the same names: fct_guest_spending
# Please print your final result or dataframe
park_experiences_list = fct_guest_spending['park_experience_type'].unique().tolist()
fct_guest_spending['year_month']= fct_guest_spending['visit_date'].dt.strftime("%Y-%m")
required_data=fct_guest_spending[fct_guest_spending['year_month']=='2024-07']
final_data= required_data.groupby(['guest_id','park_experience_type']).agg(total_visits=('park_experience_type','count'),
                                                               total_amount_spent=('amount_spent','sum')).reset_index()
final_data= final_data.groupby('park_experience_type').agg(total_visits_july=('total_visits','sum'),
                                                     total_amount_spent_july=('total_amount_spent','sum')).reset_index()
final_data['avg_spend'] = np.round(final_data['total_amount_spent_july'] / final_data['total_visits_july'], 3)
final_data_1= final_data[['park_experience_type','avg_spend']]
final_data_1= final_data_1.set_index('park_experience_type')
final_data_1 = final_data_1.reindex(park_experiences_list, fill_value=0.0).reset_index()
final_data_1

## Question 2

For guests who visited our parks more than once in August 2024, what is the difference in spending between their first and their last visit? This investigation, using sequential analysis, will reveal any shifts in guest spending behavior over multiple visits.

In [ ]:
# Step 1: Filter August 2024 visits only
required_data = fct_guest_spending[fct_guest_spending['visit_date'].dt.strftime('%Y-%m') == '2024-08']

# Step 2: Keep only guests with more than one visit
visit_counts = required_data['guest_id'].value_counts()
multi_visitors = visit_counts[visit_counts > 1].index
required_data = required_data[required_data['guest_id'].isin(multi_visitors)]

# Step 3: Sort by guest and visit date
required_data = required_data.sort_values(['guest_id', 'visit_date']).reset_index(drop=True)

# Step 4: Rank visits
required_data['visit_rank'] = required_data.groupby('guest_id').cumcount() + 1

# Step 5: Get first and last visits per guest
first_visits = required_data[required_data['visit_rank'] == 1]
last_visits = required_data.loc[required_data.groupby('guest_id')['visit_rank'].idxmax()]

# Step 6: Merge first and last visit data
visit_data = first_visits.merge(
    last_visits[['guest_id', 'amount_spent', 'visit_date']],
    on='guest_id',
    how='left',
    suffixes=('_first', '_last')
)

# Step 7: Calculate spending difference
visit_data['amount_diff'] = abs(visit_data['amount_spent_last'] - visit_data['amount_spent_first'])

# Final Output
visit_data[['guest_id', 'amount_spent_first', 'amount_spent_last', 'amount_diff']]

## Question 3

In September 2024, how can guests be categorized into distinct spending segments such as Low, Medium, and High based on their total spending? Use the following thresholds for categorization: 
-Low: Includes values from $0 up to, but not including, $50.
-Medium: Includes values from $50 up to, but not including, $100.
-High: Includes values from $100 and above. 
Exclude guests who did not make any purchases in the period.

In [ ]:
sep_data = fct_guest_spending[fct_guest_spending['visit_date'].dt.strftime('%Y-%m') == '2024-09']
guest_spending = sep_data.groupby('guest_id', as_index=False).agg(total_money_spent=('amount_spent','sum'))
guest_spending = guest_spending[guest_spending['total_money_spent'] > 0]
guest_spending['spending_category'] = np.where(
    guest_spending['total_money_spent'] < 50, 
    'Low',
    np.where(
        guest_spending['total_money_spent'] < 100,
        'Medium',
        'High'
    )
)

# 5. Final result
guest_spending[['guest_id', 'total_money_spent', 'spending_category']]

Made with ❤️ by [Interview Master](https://www.interviewmaster.ai)